In [88]:
import pandas as pd
import numpy as np 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [41]:
data=pd.read_csv("train/train.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [42]:
data.dropna(inplace=True)
data.reset_index(inplace=True)

Drop all nan values 


In [43]:
message=data.copy()

## Data Cleaning

The data we have in raw format for traing the data with the enbedding layer we have to conver 
the data into the vector

In [44]:
ps=PorterStemmer()
final_message=[]
for sentance in range(0,len(message)):
    review=re.sub('[^a-zA-Z]',' ',message["title"][sentance])
    review=review.lower()
    review=review.split()
    
    review=(ps.stem(word) for word in review if word not in stopwords.words("english"))
    review=" ".join(review)
    final_message.append(review)

converting the the text data into one hot in coding

In [45]:
one_hot_rep=[one_hot(word,5000) for word in final_message ]

In [46]:
one_hot_rep

[[1400, 1377, 3525, 3318, 923, 1608, 2016, 3698, 4486, 1633],
 [1073, 2505, 656, 4028, 3893, 1882, 1928],
 [3380, 938, 2088, 1645],
 [3882, 2490, 4017, 3612, 3711, 2201],
 [1583, 3893, 2383, 1723, 2605, 1203, 3893, 2952, 2340, 3105],
 [1871,
  1297,
  3985,
  2372,
  1697,
  1308,
  2217,
  3676,
  4699,
  1744,
  1822,
  2306,
  4410,
  4865,
  1928],
 [3019, 749, 962, 1276, 976, 2864, 1793, 194, 785, 1120, 1879],
 [3205, 3765, 4387, 1091, 67, 1380, 1308, 4753, 785, 1120, 1879],
 [3194, 3100, 3154, 2109, 2130, 4387, 1530, 3284, 1308, 4055],
 [2511, 1121, 1500, 4250, 1450, 2709, 2565, 603],
 [212, 1202, 81, 2821, 3060, 3078, 820, 723, 1357, 251, 3866],
 [3612, 1508, 923, 4387, 1308, 67],
 [2490, 327, 955, 2385, 2182, 3407, 4870, 4142, 1353],
 [3355, 1706, 3895, 1566, 3811, 3429, 4200, 785, 1120, 1879],
 [3124, 2907, 1804, 3602, 323, 785, 1120, 1879],
 [1308, 4515, 2461, 1740, 1622, 1121, 1574, 2987, 4426, 4077],
 [4350, 4904, 2505],
 [2988, 3162, 3144, 188, 1308, 3766, 247, 1928],
 [24

The Embadding layer require the same vector size so add in pre padding as 0 to the vectore

In [70]:
embedding_titles=pad_sequences(one_hot_rep,maxlen=20)
embedding_titles.shape

(18285, 20)

Now the data is ready to aply the model

## Model Building

In [95]:
def create_model(layers,activation,kernal):
    model=Sequential()
    embedding_vector_features=40
    voc_size=5000
    model.add(Embedding(voc_size,embedding_vector_features,input_length=20))
    model.add(LSTM(units=layers,activation=activation, kernel_initializer=kernal))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    return model

In [109]:
model=KerasClassifier(create_model,verbose=0)
layers=[(100),(20,40)]
activation=['tanh','softmax','relu']
kernal=["glorot_uniform","glorot_normal"]
param_grid=dict(layers=layers,activation=activation,kernal=kernal)
grid=GridSearchCV(estimator=model,param_grid=param_grid)

In [104]:
x=np.array(embedding_titles)
y=data["label"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=40)

In [110]:
grid_result=grid.fit(x_train,y_train)

In [108]:
grid_result.best_params_


{'activation': 'tanh', 'kernal': 'glorot_uniform', 'layers': 100}

In [84]:
print(confusion_matrix(y_test,y_predict))

[[1870  187]
 [ 163 1437]]


In [85]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      2057
           1       0.88      0.90      0.89      1600

    accuracy                           0.90      3657
   macro avg       0.90      0.90      0.90      3657
weighted avg       0.90      0.90      0.90      3657

